# 模型的评估
除了使用estimator的score函数简单粗暴地评估模型的质量之外，在sklearn.metrics模块针对不同的问题提出了各种评价指标。使用sklearn.model_selection模块中的交叉验证相关方法可以评估模型的泛化能力，能有有效地避免过拟合。
## 一、metrics评估模型综述
sklearn.metrics中评估指标有两类：以_score结尾，越大越好；以_error或_loss结尾，越小越好。<br>
常用的分类评估指标;acuracy_score、f1_score、precision_score、recall_score等。<br>
常用的回归评估指标包括：r2_score、explained_variance_score 等。<br>
常用的聚类评估指标包括：adjusted_rand_score、adjusted_mutual_info_score等。<br>
### 1.分类模型的评估<br>
confusion_matrix 混淆矩阵，误差矩阵。<br>
TP:真正例          FN:假负例 <br>
FP:假正例          TN:真负例<br><br>
模型整体分类效果评估：<br>
accuracy正确率：（TP+TN）/(TP+TN+FP+FN)<br>
模型对某种类别的分类效果：<br>
precision精确率：也叫查准率，不把正样本标错的能力。公式：TP/(TP+FP)<br>
recall召回率：也叫查全率，识别全部正样本的能力。公式：TP/(TP+FN)<br>
f1_score：精确率和召回率的调和平均值。公式：2PR/（P+R）<br><br>
利用不同方式将类别效果进行求和平均得到整体分类效果：<br>
macro_avergaed:宏平均，每种类别预测的效果一样重要。计算方法：统计各个类标的TP、FP、FN、TN，分别计算各自的Precision和Recall，得到各自的F1值，然后取平均值得到macro-F1。<br>
micro_averaged:微平均，每一次分类预测的效果一样重要。计算方法：统计各个类标的TP、FP、FN、TN，加和构成新的TP、FP、FN、TN，然后即可计算micro-Precision和micro-Recall，得到micro-F1。<br>
weighted_averaged：加权平均，每种类别预测的效果与该样本出现的频率成正比。<br><br>
仅适用于概率模型且问题为二分类问题的评估方法：<br>
ROC曲线、auc_score<br>

In [138]:
from sklearn import metrics   #引入metrics模块
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

iris=datasets.load_iris()
cal=SVC(kernel='linear')
x_train,x_test,y_train,y_test=train_test_split(iris.data,iris.target,test_size=1/3)
cal.fit(x_train,y_train)
y_predict=cal.predict(x_test)
print(metrics.confusion_matrix(y_test,y_predict))
print("准确率：",metrics.accuracy_score(y_test,y_predict))
print("精度（查准率）：",metrics.precision_score(y_test,y_predict,average=None))#输出每个类别的查准率
print("召回率（查全率）：",metrics.recall_score(y_test,y_predict,average=None))#输出每个类别的查全率
print("F-score:",metrics.f1_score(y_test,y_predict,average=None))#输出每个类别f1_score
print("精度（查准率）：",metrics.precision_score(y_test,y_predict,average='macro'))#输出每个类别的查准率
print("召回率（查全率）：",metrics.recall_score(y_test,y_predict,average='micro'))#输出每个类别的查全率
print("F-score:",metrics.f1_score(y_test,y_predict,average='weighted'))#输出每个类别f1_score

[[18  0  0]
 [ 0 13  1]
 [ 0  0 18]]
准确率： 0.98
精度（查准率）： [1.         1.         0.94736842]
召回率（查全率）： [1.         0.92857143 1.        ]
F-score: [1.         0.96296296 0.97297297]
精度（查准率）： 0.9824561403508771
召回率（查全率）： 0.98
F-score: 0.9798998998998999


# 2.回归模型的评估
回归模型常用的评估指标有：<br>
r2_score=1-$\frac{\sum_i(y_i-\hat y_i)}{\sum_i(y_i-\bar y)}$ <br>
explained_variance_score=1-$\frac{Var(y_i-\hat y_i)}{Var(y_i)}$

In [139]:
from sklearn import datasets
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

boston=datasets.load_boston()

scaler=MinMaxScaler(feature_range=(0,1))
scaler.fit_transform(boston.data)

x_train,x_test,y_train,y_test=train_test_split(boston.data,boston.target,test_size=1/3)

reg=DecisionTreeRegressor()
reg.fit(x_train,y_train)
y_predict=reg.predict(x_test)

print("R^2:",metrics.r2_score(y_predict,y_test))
print("explained_variance_score:",metrics.explained_variance_score(y_predict,y_test))


R^2: 0.7541132896183421
explained_variance_score: 0.7553234228617665


### 3.是用虚拟估计器产生基准得分
对于监督学习（分类和回归）、可以采用一些基于经验的简单估计策略（虚拟估计）的得分作为参照基准值。
DummyClassifier 实现了几种简单的分类策略:<br><br>
stratified　　　　　　 通过在训练集类分布方面来生成随机预测<br>
most_frequent　　　　　 总是预测训练集中最常见的标签<br>
prior 　　　　　　类似most_frequenct，但具有precit_proba方法<br>
uniform 　　　　　　　　　　　随机产生预测<br>
constant 　　　　　　　总是预测用户提供的常量标签<br><br>
DummyRegressor 实现了四个简单的经验法则来进行回归:<br><br>
mean 　　　　　　　　　总是预测训练目标的平均值<br>
median 　　　　　　　　总是预测训练目标的中位数<br>
quantile 　　　总是预测用户提供的训练目标的 quantile（分位数）<br>
constant　　　　　　　　 总是预测由用户提供的常数值<br>

In [140]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier

In [141]:
iris=datasets.load_iris()

x_train,x_test,y_train,y_test=train_test_split(iris.data,iris.target,test_size=1/3)

dummy=DummyClassifier(strategy='stratified')
svc=SVC(kernel='linear')

dummy.fit(x_train,y_train)
svc.fit(x_train,y_train)

dummy.predict(x_test)
svc.predict(x_test)

print("dummy_score:",dummy.score(x_test,y_test))
print("svc_score:",svc.score(x_test,y_test))



dummy_score: 0.28
svc_score: 1.0


### 4.交叉验证
交叉验证法的基本思想：以不同方式多次将数据集划分为训练集和测试集，分别测试和训练，在综合最后的测试得分。每个数据点在一些划分情况下属于训练集，在另外一些情况下属于测试集。<br>
常用的交叉验证方法：<br>
K折(KFold),留一交叉验证(LeaveOneOut,LOO),留P交叉验证(LeavePOut,LPO),随机排列交叉验证(ShuffleSplit)。此外，为了保证训练集中每种标签类别数据的分布和完整数据集中的分布一致，可以采用分层交叉<br>
验证方法(StratifiedKFold，StratifiedShuffleSplit)。<br>

In [142]:
#随机排列交叉验证
from sklearn.model_selection import ShuffleSplit
import numpy as np

x=np.arange(20)

ss=ShuffleSplit(n_splits=10,test_size=0.3)
for train_index,test_index in ss.split(x):     #输入为标签为索引
    print("%s%s"%(train_index,test_index))

[19 15 17 13 11  1  7  0  8  4 14 10  9  5][18  3 16  2 12  6]
[17 13 18 12 19  3 11  1  4  7 14 15  9  0][10  5  2 16  8  6]
[ 5  4  1  7 14 18  8 15 19  0 11 16  9  6][10  2  3 12 17 13]
[ 6 10  7  5  4  1 18 15 16 11  2 17 13  3][14 12  9  8 19  0]
[ 9  7 15 18 10 12 13 17  3  1  8 11  0 14][19  2 16  6  4  5]
[ 0 15  7  5 11 17 13  8 19 12  4 14  3  2][16  9  6 10 18  1]
[ 2  8  9  4 16  5 19 17  7  1 12  6 14  0][11  3 10 13 15 18]
[19 10  6  9  0 11  8 16  4  2 15 14 12  5][13  7  1 17  3 18]
[ 0  1  8  3  4 18 15 12  5 17  7 11  2 13][14  6 19 10  9 16]
[ 6  7 17 11 12  0 19 16 18 15  8  4  2  1][ 3 14 13  5  9 10]


In [143]:
#分层K折交叉验证
from sklearn.model_selection import StratifiedKFold
import numpy as np

x=np.ones(12)
y=list('000011111111')

skf=StratifiedKFold(n_splits=3)
for train_index,test_index in skf.split(x,y):     #输入为标签为索引
    print("%s%s"%(train_index,test_index))

[ 2  3  7  8  9 10 11][0 1 4 5 6]
[ 0  1  3  4  5  6 10 11][2 7 8 9]
[0 1 2 4 5 6 7 8 9][ 3 10 11]


In [144]:
# 留P分组交叉验证
from sklearn.model_selection import LeavePGroupsOut
X = np.arange(9)
y = [1,1,1,2,2,2,2,2,2]
groups = [1,2,1,2,2,3,3,1,3]#定义一种分组方式
lpgo = LeavePGroupsOut(n_groups=2)#n_groups=2表示从分组中选两组作为训练集
for train_index, test_index in lpgo.split(X, y,groups=groups):
    print("%s %s" % (train_index, test_index))

[5 6 8] [0 1 2 3 4 7]
[1 3 4] [0 2 5 6 7 8]
[0 2 7] [1 3 4 5 6 8]


### 5.交叉验证综合评分
交叉验证综合评分：<br>
调用 cross_val_score 函数可以计算模型在各交叉验证数据集上的得分。<br>
可以指定metrics中的打分函数，也可以指定交叉验证迭代器。<br>

In [145]:
#使用cross_val_score函数
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

bc=datasets.load_breast_cancer()
svc=SVC(kernel='rbf',C=1)
score=cross_val_score(svc,bc.data,bc.target,cv=10)
print("%2.2f"%score.mean())

d:\py\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
d:\py\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
d:\py\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
d:\py\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version

0.63


d:\py\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
d:\py\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
d:\py\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
d:\py\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version

In [146]:
#改变评分方法
score=cross_val_score(svc,bc.data,bc.target,cv=10,scoring='f1_macro')
print("%2.2f"%score.mean())

d:\py\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
d:\py\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
d:\py\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
d:\py\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, wa

0.39


d:\py\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
d:\py\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
d:\py\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
d:\py\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, wa

In [147]:
#改变交叉验证策略
from sklearn.model_selection import ShuffleSplit,StratifiedKFold
ss=ShuffleSplit(n_splits=5)
ss1=StratifiedKFold(n_splits=5)
score=cross_val_score(svc,bc.data,bc.target,cv=ss1)
print("%2.2f"%score.mean())

d:\py\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
d:\py\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
d:\py\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
d:\py\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version

0.63


In [155]:
#使用cross_validate,基本用法同cross_val_score，比起更高级
from sklearn.model_selection import cross_validate
#from sklearn.metrics import recall_score
clf = SVC(kernel='linear', C=1, random_state=0)
scores = cross_validate(clf, iris.data, iris.target, scoring=['f1_macro','f1_micro'],#返回的是一个字典
                        cv=10, return_train_score=False)
print(scores)  #打印这个字典
#scores['test_recall_macro'] 
print(scores['fit_time']) #训练集拟合时间
print(scores['score_time'])#测试集评分时间
print('f1_micro:',scores['test_f1_micro'])
print('f1_macro:',scores['test_f1_macro'])

{'fit_time': array([0.00199366, 0.0009985 , 0.00099683, 0.00099802, 0.0009985 ,
       0.00099707, 0.00098538, 0.        , 0.00099874, 0.0010047 ]), 'score_time': array([0.00199676, 0.00099754, 0.00199199, 0.00199485, 0.00199199,
       0.00305128, 0.00099707, 0.00296712, 0.0020268 , 0.0019865 ]), 'test_f1_macro': array([1.        , 0.93265993, 1.        , 1.        , 0.86111111,
       1.        , 0.93265993, 1.        , 1.        , 1.        ]), 'test_f1_micro': array([1.        , 0.93333333, 1.        , 1.        , 0.86666667,
       1.        , 0.93333333, 1.        , 1.        , 1.        ])}
[0.00199366 0.0009985  0.00099683 0.00099802 0.0009985  0.00099707
 0.00098538 0.         0.00099874 0.0010047 ]
[0.00199676 0.00099754 0.00199199 0.00199485 0.00199199 0.00305128
 0.00099707 0.00296712 0.0020268  0.0019865 ]
f1_micro: [1.         0.93333333 1.         1.         0.86666667 1.
 0.93333333 1.         1.         1.        ]
f1_macro: [1.         0.93265993 1.         1.        